In [136]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nibabel as nib
import pandas as pd
from torchvision import transforms
from tqdm import tqdm
import os
import datetime
import pandas as pd
import numpy as np
import nibabel as nib
import torch
from torch.utils.data import Dataset
import random

In [137]:
# Set seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [138]:
import importlib.util
import sys

# Specify the full path to the module file
module_path = 'D:\\Github Folder\\MasterThesis\\Code\\FUNCTIONS.py'

# Load the module
spec = importlib.util.spec_from_file_location("FUNCTIONS", module_path)
functions = importlib.util.module_from_spec(spec)
spec.loader.exec_module(functions)

# Now you can use the functions as if you had imported them
load_datasets = functions.load_datasets
create_dataloaders = functions.create_dataloaders
train_and_validate = functions.train_and_validate
test_model = functions.test_model


In [139]:
df = pd.read_excel(r"references\NEW_COMBINED_FINAL_Subject_info.xlsx")

In [140]:
# Assuming 'df' is your DataFrame loaded with the 'Research Group' column available
label_categories = pd.Categorical(df['Research Group'])
label_mapping = {code: category for code, category in enumerate(label_categories.categories)}

In [141]:
class VGG3D(nn.Module):
	def __init__(self, num_classes=1, input_shape=(1,110,110,110)): # input: input_shape:	[num_of_filters, kernel_size] (e.g. [256, 25])
		super(VGG3D, self).__init__()
		self.conv1 = nn.Sequential(
			nn.Conv3d(
				in_channels=input_shape[0],        # input height
				out_channels=8,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1
			),
            nn.ReLU(),
            nn.Conv3d(
				in_channels=8,        # input height
				out_channels=8,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1              
            ),
			nn.ReLU(),                  # activation
			nn.MaxPool3d(kernel_size=(2,2,2), stride=2) # choose max value in 2x2 area
		)
        
		self.conv2 = nn.Sequential(
			nn.Conv3d(
				in_channels=8,        # input height
				out_channels=16,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1              
			),
            nn.ReLU(),
            nn.Conv3d(
				in_channels=16,        # input height
				out_channels=16,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1                
			),
			nn.ReLU(),                  # activation
			nn.MaxPool3d(kernel_size=(2,2,2), stride=2) # choose max value in 2x2 area
		)
        
		self.conv3 = nn.Sequential(
			nn.Conv3d(
				in_channels=16,        # input height
				out_channels=32,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1               
			),
            nn.ReLU(),
            nn.Conv3d(
				in_channels=32,        # input height
				out_channels=32,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1                
			),
			nn.ReLU(),                  # activation
            nn.Conv3d(
				in_channels=32,        # input height
				out_channels=32,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1              
			),
			nn.ReLU(),
			nn.MaxPool3d(kernel_size=(2,2,2), stride=2) # choose max value in 2x2 area
		)

		self.conv4 = nn.Sequential(
			nn.Conv3d(
				in_channels=32,        # input height
				out_channels=64,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1                
			),
            nn.ReLU(),
            nn.Conv3d(
				in_channels=64,        # input height
				out_channels=64,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1                
			),
			nn.ReLU(),                  # activation
            nn.Conv3d(
				in_channels=64,        # input height
				out_channels=64,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1
			),
			nn.ReLU(),
			nn.MaxPool3d(kernel_size=(2,2,2), stride=2) # choose max value in 2x2 area
		)
		
		fc1_output_features=128     
		self.fc1 = nn.Sequential(
			 nn.Linear(13824, 128),
 			 nn.BatchNorm1d(128),            
			 nn.ReLU()
		 )

		fc2_output_features=64           
		self.fc2 = nn.Sequential(
			 nn.Linear(fc1_output_features, fc2_output_features),
			 nn.BatchNorm1d(fc2_output_features),
			 nn.ReLU()
		 )

		self.out = nn.Linear(fc2_output_features, num_classes)  # num_classes could be 1 for binary or more for multi-class

	def forward(self, x, drop_prob=0.8):

		x = self.conv1(x)
#		print(x.shape)        
		x = self.conv2(x)
#		print(x.shape)        
		x = self.conv3(x)
#		print(x.shape)        
		x = self.conv4(x)
#		print(x.shape)        
		x = x.view(x.size(0), -1)           # flatten the output of conv2 to (batch_size, num_filter * w * h)
#		print(x.shape)        
		x = self.fc1(x)
		x = nn.Dropout(drop_prob)(x)
		x = self.fc2(x)
		#x = nn.Dropout(drop_prob)(x)        
		prob = self.out(x) # probability
# 		y_hat = self.out_act(prob) # label
# 		return y_hat, prob, x    # return x for visualization
		return prob

In [142]:
def select_optimizer(model, config):
    """Select optimizer based on configuration."""
    lr = config.get('lr', 0.001)  # Default learning rate
    weight_decay = config.get('weight_decay', 0)  # Default weight decay
    
    if config['optimizer'] == 'Adam':
        return optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif config['optimizer'] == 'SGD':
        momentum = config.get('momentum', 0.9)  # Default momentum for SGD
        return optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    else:
        raise ValueError("Unsupported optimizer")

def select_criterion(config):
    """Select loss criterion based on configuration."""
    if config['loss_criterion'] == 'Cross-Entropy':
        return nn.CrossEntropyLoss()
    elif config['loss_criterion'] == 'BCEWithLogits':
        return nn.BCEWithLogitsLoss()
    else:
        raise ValueError("Unsupported loss criterion")
    
    

In [156]:
from openpyxl import load_workbook
import pandas as pd
import os
import datetime
import torch
import torch.nn as nn
import torch.optim as optim






def run_experiment(df, config):
    train_dataset, val_dataset, test_dataset = load_datasets(df, image_type = config['image_type'], loss_type = config['loss_criterion'])
    train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=config['batch_size'])
    
    device = torch.device("cuda")
    model = VGG3D().to(device)
    
    criterion = select_criterion(config)
    optimizer = select_optimizer(model, config)


    # Training and validation
    train_accuracies, val_accuracies, val_losses, results_df = train_and_validate(model, train_loader, val_loader, criterion, optimizer, config['num_epochs'], config['patience'], device, config['loss_criterion'])
    test_results, test_accuracy = test_model(model, test_loader, label_mapping, device, loss_type = config['loss_criterion'])
    
    # Save detailed results to Excel
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime('%Y-%m-%d_%H-%M-%S')
    report_filename = os.path.join('reports', f'{formatted_time}_Experiment.xlsx')
    onnx_filename = os.path.join('models', f'{formatted_time}_Model.')
          # Save the model to ONNX


    
    summary_data = {
        'Phase': ['Training', 'Validation', 'Testing'],
        'Accuracy': [train_accuracies[-1], val_accuracies[-1], test_accuracy]
    }
    summary_df = pd.DataFrame(summary_data)
    all_results = pd.DataFrame(test_results)
    config_df = pd.DataFrame([config])
    
    with pd.ExcelWriter(report_filename) as writer:
        config_df.to_excel(writer, sheet_name='Configuration')
        all_results.to_excel(writer, sheet_name='Results')
        summary_df.to_excel(writer, sheet_name='Summary')
        results_df.to_excel(writer, sheet_name='Training_Results')

# Append a summary of this experiment to the cumulative RESULTS.xlsx file
    results_file = os.path.join('reports', 'RESULTS.xlsx')
    experiment_summary = {**config, **{'Training Accuracy': train_accuracies[-1], 'Validation Accuracy': val_accuracies[-1], 'Test Accuracy': test_accuracy, 'DATETIME': formatted_time}}
    summary_row = pd.DataFrame([experiment_summary])

    if os.path.exists(results_file):
        with pd.ExcelWriter(results_file, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            existing_df = pd.read_excel(results_file)
            combined_df = pd.concat([existing_df, summary_row], ignore_index=True)
            combined_df = combined_df.reindex(columns=(existing_df.columns.tolist() + [col for col in summary_row.columns if col not in existing_df.columns]))
            combined_df.to_excel(writer, index=False, sheet_name='Sheet1')
    else:
        summary_row.to_excel(results_file, index=False)

    # Save the model to ONNX
    dummy_input = torch.randn(1, 1, 110, 110, 110, device=device)  # Adjust size according to your model's input
    torch.onnx.export(model, dummy_input, onnx_filename, export_params=True)

    return report_filename, train_accuracies[-1], val_accuracies[-1], test_accuracy


# Example configuration and use case
config = {
    'optimizer': 'SGD',
    'lr' : 0.005,
    'loss_criterion': 'BCEWithLogits',
    'num_epochs': 150,
    'batch_size': 8,
    'patience': 20,
    'Description' : 'TEST',
    'image_type' : 'Resampled Images_fused',
    'weight_decay' : 0,
    'momentum' : 0.7

}

In [146]:
run_experiment(df, config)

Epoch 1/150 - Train:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch 1/150 - Train: 100%|██████████| 39/39 [00:12<00:00,  3.10it/s]


Epoch 1: Train Loss: 0.7173 - Train Accuracy: 49.36%


Epoch 1/150 - Validate: 100%|██████████| 5/5 [00:00<00:00,  6.46it/s]


Epoch 1: Validation Loss: 0.7762 - Validation Accuracy: 45.00%


...

Epoch 92: Train Loss: 0.4958 - Train Accuracy: 81.09%


Epoch 92/150 - Validate: 100%|██████████| 5/5 [00:00<00:00,  6.75it/s]


Epoch 92: Validation Loss: 1.5212 - Validation Accuracy: 57.50%
Early stopping triggered after 92 epochs due to no improvement in validation loss or accuracy.


Testing: 100%|██████████| 11/11 [00:01<00:00,  6.83it/s]
c:\Users\Micha\.pyenv\pyenv-win\versions\3.10.10\lib\site-packages\torch\onnx\symbolic_helper.py:1515: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'dropout' is set to train=True. Exporting with train=True.
  warnings.warn(


('reports\\2024-05-14_13-33-27_Experiment.xlsx',
 81.08974358974359,
 57.5,
 40.90909090909091)

# RUN

In [158]:
  #column_mapping = {
  #          'Co-registered PET': 'Co-registered PET',
   #         'Fused Images': 'Fused Images',
    #        'Masked PET': 'Masked PET',
     #       'Spatial Normalization': 'Spatial Normalization',
      #      'Resampled Images(Co-registered PET)': 'Resampled Images(Co-registered PET)',
       #     'Resampled Images(Masked PET)': 'Resampled Images(Masked PET)',
        #    'Resampled Images(Spatial Normalization)': 'Resampled Images(Spatial Normalization)',
         #   'Resampled Images_fused': 'Resampled Images_fused'
        #}

image_types = ['Resampled Images_fused','Resampled Images(Co-registered PET)', 'Resampled Images(Masked PET)', 'Resampled Images(Spatial Normalization)']
results = []
for image_type in image_types:
    config['image_type'] = image_type
    print(f"Working on image type: {image_type}")
    result = run_experiment(df, config)
    results.append(result)



Working on image type: Resampled Images_fused


Epoch 1/150 - Train:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch 1/150 - Train: 100%|██████████| 39/39 [00:25<00:00,  1.55it/s]


Epoch 1: Train Loss: 0.7171 - Train Accuracy: 46.79%


Epoch 1/150 - Validate: 100%|██████████| 5/5 [00:01<00:00,  3.58it/s]


...

Epoch 26/150 - Train: 100%|██████████| 39/39 [00:21<00:00,  1.80it/s]


Epoch 26: Train Loss: 0.7038 - Train Accuracy: 46.79%


Epoch 26/150 - Validate: 100%|██████████| 5/5 [00:01<00:00,  2.54it/s]


Epoch 26: Validation Loss: 0.6766 - Validation Accuracy: 55.00%
Early stopping triggered after 26 epochs due to no improvement in validation loss or accuracy.


Testing: 100%|██████████| 11/11 [00:05<00:00,  2.06it/s]
c:\Users\Micha\.pyenv\pyenv-win\versions\3.10.10\lib\site-packages\torch\onnx\symbolic_helper.py:1515: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'dropout' is set to train=True. Exporting with train=True.
  warnings.warn(


In [150]:
import pandas as pd
from tqdm import tqdm
import sys
import os



def run_multiple_experiments(df):
    """Run experiments for all combinations of configurations."""
    optimizers = ['SGD', 'Adam']
    loss_criterions = ['BCEWithLogits']
    learning_rates = [0.001, 0.05, 0.01, 0.1]
    image_types = [
        'Resampled Images_fused','Resampled Images(Co-registered PET)', 'Resampled Images(Masked PET)']

    num_epochs = 200
    patience = 20
    batch_size = 16

    results = []
    total = len(optimizers) * len(loss_criterions) * len(learning_rates) * len(image_types)
    progress_bar = tqdm(total=total, desc='Running Experiments', leave=True)

    for lr in learning_rates:    
        for optimizer in optimizers:
            for loss_criterion in loss_criterions:
                    for image_type in image_types:
                        config = {
                            'optimizer': optimizer,
                            'loss_criterion': loss_criterion,
                            'lr': lr,
                            'num_epochs': num_epochs,
                            'batch_size': batch_size,
                            'patience': patience,
                            'image_type': image_type,
                            'Description': 'Overnight test of learning rates, optimizer and loss criterions'
                        }
                        
                    report_filename, train_acc, val_acc, test_acc = run_experiment(df, config)

                    results.append({
                        'Optimizer': optimizer,
                        'Loss Criterion': loss_criterion,
                        'Learning Rate': lr,
                        'Image Type': image_type,
                        'Report File': report_filename,
                        'Train Accuracy': train_acc,
                        'Validation Accuracy': val_acc,
                        'Test Accuracy': test_acc
                    })
                    progress_bar.update(1)

    progress_bar.close()

    # Convert results to DataFrame and save or return
    results_df = pd.DataFrame(results)
    results_df.to_csv('experiment_results_summary.csv', index=False)
    return results_df


In [151]:
results_df = run_multiple_experiments(df)

Epoch 1/200 - Train: 100%|██████████| 19/19 [00:18<00:00,  1.00it/s]


Epoch 1: Train Loss: 0.7343 - Train Accuracy: 47.04%


Epoch 1/200 - Validate: 100%|██████████| 2/2 [00:01<00:00,  1.77it/s]


Epoch 1: Validation Loss: 0.6671 - Validation Accuracy: 62.50%


Epoch 2/200 - Train: 100%|██████████| 19/19 [00:13<00:00,  1.36it/s]


...

Epoch 22/200 - Train: 100%|██████████| 19/19 [00:11<00:00,  1.59it/s]


Epoch 22: Train Loss: 0.6946 - Train Accuracy: 49.67%


Epoch 22/200 - Validate: 100%|██████████| 2/2 [00:00<00:00,  2.98it/s]


Epoch 22: Validation Loss: 0.6952 - Validation Accuracy: 40.62%
Early stopping triggered after 22 epochs due to no improvement in validation loss or accuracy.


Testing: 100%|██████████| 5/5 [00:01<00:00,  3.63it/s]
c:\Users\Micha\.pyenv\pyenv-win\versions\3.10.10\lib\site-packages\torch\onnx\symbolic_helper.py:1515: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'dropout' is set to train=True. Exporting with train=True.
  warnings.warn(
Running Experiments:  33%|███▎      | 8/24 [46:07<1:32:15, 345.98s/it]
